In [2]:
from telnetlib import X3PAD
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pickle 
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
#from keras.wrappers.scikit_learn import KerasRefressor
# from keras.layers import Dense 
# from keras.models import Sequential
import pickle 
from sklearn.metrics import mean_squared_error

In [3]:
df_2020 = pd.read_csv('merged_gw2020.csv',low_memory = False)
df_2021 = pd.read_csv('merged_gw2021.csv',low_memory = False)


In [4]:
season_2020 = pd.DataFrame({'season' : ['2019/20']*df_2020.shape[0]   })
season_2021 = pd.DataFrame({'season' : ['2020/21']*df_2021.shape[0]   })

In [5]:
df_2020 =pd.concat([df_2020, season_2020] ,axis = 'columns')
df_2021 =pd.concat([df_2021, season_2021] ,axis = 'columns')


In [6]:
df= pd.concat([df_2020 , df_2021], axis = 'index',ignore_index = True)



In [7]:
df["was_home"] = df["was_home"].astype(int)


In [8]:

df.sort_values(by=['name','season','GW'], inplace =True)
df.reset_index(inplace = True)


df.drop(['index'],axis= 'columns' , inplace =True)

In [9]:
df['id'] = df.groupby(['name']).ngroup()

In [10]:
df['pos']= df.groupby(['position']).ngroup()

In [11]:
df['teams'] = df.groupby(['team']).ngroup()

In [12]:
identity = df['id'].unique()


### points milaune thau


In [13]:
#enter value of x where x is the latest gameweek
x = 25

In [14]:
filt = (df['GW'] == x) & (df['season']=='2020/21')
df_points = df.loc[filt , ['total_points','id' ]]
df_points.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_points.reset_index(inplace = True)


df_points.drop(['index'],axis= 'columns' , inplace =True)
df_points

,total_points,id
0,0,0
1,0,1
2,2,2
3,0,5
4,0,6
...,...,...
627,9,956
628,1,957
629,0,959
630,0,962


In [15]:
y=x-1
Gameweeks=[]

In [16]:
for i in range(5):
    Gameweeks.append(y)
    y=y-1


In [17]:
dataframes = {}

In [18]:
for GW in Gameweeks:
    filt=(df['GW'] == GW) & (df['season'] == '2020/21')
    dataframes[GW] = df[filt]
    dataframes[GW].reset_index(inplace = True)
    dataframes[GW].drop(['index'],axis= 'columns' , inplace =True)
    dataframes[GW].drop([ 'kickoff_time','season', 'name' , 'position','pos' ,'GW' ,'team','teams'],axis= 'columns' , inplace =True)
    

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [19]:
dataframes[24].columns

Index(['xP', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'id'],
      dtype='object')

In [20]:
n = 0
for keys in dataframes:
    column = []
    
    for value in dataframes[keys].columns:
        
        if (value != 'id')  & (value!='team'):
            value = value +f'_{n}' 
        column.append(value)
        
    dataframes[keys].columns = column
    n=n-1       

In [21]:
for keys in dataframes:
    dataframes[keys].drop_duplicates(subset='id' , keep ='first' , inplace = True)
    dataframes[keys].reset_index(inplace = True)
    dataframes[keys].drop(['index'],axis= 'columns' , inplace =True)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [22]:
#for keys in dataframes:
#    print(dataframes[keys].columns)

In [23]:

df1 = pd.merge(dataframes[x-1] , dataframes[x-2], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-3], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-4], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-5], on = 'id' , how = 'outer')


In [24]:
filt = (df['GW'] == x-1) | (df['GW'] == x-2) | (df['GW'] == x-3) | (df['GW'] == x-4) | (df['GW'] == x-5)
df_position = df.loc[filt , ['id' , 'pos','teams']]

In [25]:
df_position.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_position.reset_index(inplace = True)
df_position.drop(['index'],axis= 'columns' , inplace =True)

In [26]:
df1.drop_duplicates(subset='id',keep = 'first',inplace = True)
df1.reset_index(inplace = True)
df1.drop(['index'],axis= 'columns' , inplace =True)

In [27]:
df1 = pd.merge(df1 , df_position, on = 'id' , how = 'inner')

In [28]:
filt = df['GW']== x 
df_points = df.loc[filt,['id' , 'total_points']]
df_points.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_points.reset_index(inplace = True)
df_points.drop(['index'],axis= 'columns' , inplace =True)

In [29]:
df_points

,id,total_points
0,0,0
1,1,2
2,2,2
3,3,0
4,4,3
...,...,...
911,960,0
912,961,0
913,962,0
914,963,2


In [30]:

df1 = pd.merge(df1 , df_points, on = 'id' , how = 'inner')


In [31]:
#merged =pd.concat([df1.id,df1.teams,df1.pos] ,axis = 'columns')



#df1.drop(['id' , 'teams' , 'pos'],axis= 'columns',inplace = True)


In [32]:
df['total_points'].isnull().values.any()

False

In [33]:
df1 = df1.fillna(df1.median())

In [34]:
df1

,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,element_0,fixture_0,goals_conceded_0,goals_scored_0,...,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4,pos,teams,total_points
0,6.5,0.0,2.0,30.0,1.0,25.8,411.0,235.0,0.0,0.0,...,0.0,-26734.0,479.0,27213.0,53.0,0.0,0.0,0,21,2
1,1.3,0.0,0.0,5.0,0.0,0.8,575.0,233.0,0.0,0.0,...,7.0,-3.0,25.0,28.0,49.0,0.0,0.0,3,4,2
2,9.0,0.0,2.0,31.0,1.0,1.0,559.0,236.0,0.0,0.0,...,0.0,-2.0,255.0,445.0,47.0,0.0,0.0,2,16,3
3,0.0,0.0,0.0,0.0,0.0,0.0,496.0,231.0,0.0,0.0,...,0.0,-2.0,255.0,445.0,47.0,0.0,0.0,3,1,0
4,1.0,0.0,0.0,0.0,0.0,0.0,288.0,233.0,0.0,0.0,...,2.0,-1478.0,2307.0,3785.0,51.0,1.0,0.0,0,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,2.0,4421.0,9650.0,5229.0,45.0,0.0,0.0,0,3,0
678,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,0.0,-20.0,21.0,41.0,45.0,0.0,0.0,3,3,0
679,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,0.0,-2.0,255.0,445.0,47.0,0.0,0.0,2,3,0
680,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,0.0,-3728.0,158.0,3886.0,86.0,1.0,0.0,1,5,2


In [35]:
#df1_scaled= pd.DataFrame(df1_scaled, columns = list(df1.columns))

In [36]:
#df1_scaled.info()

In [37]:
#merged.info()

In [38]:
#merged.isnull().values.any()

In [39]:
#df1_scaled.isnull().values.any()

In [40]:
#df1 = df = pd.concat([df1_scaled , merged] ,axis = 'columns')

In [41]:
df1

,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,element_0,fixture_0,goals_conceded_0,goals_scored_0,...,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4,pos,teams,total_points
0,6.5,0.0,2.0,30.0,1.0,25.8,411.0,235.0,0.0,0.0,...,0.0,-26734.0,479.0,27213.0,53.0,0.0,0.0,0,21,2
1,1.3,0.0,0.0,5.0,0.0,0.8,575.0,233.0,0.0,0.0,...,7.0,-3.0,25.0,28.0,49.0,0.0,0.0,3,4,2
2,9.0,0.0,2.0,31.0,1.0,1.0,559.0,236.0,0.0,0.0,...,0.0,-2.0,255.0,445.0,47.0,0.0,0.0,2,16,3
3,0.0,0.0,0.0,0.0,0.0,0.0,496.0,231.0,0.0,0.0,...,0.0,-2.0,255.0,445.0,47.0,0.0,0.0,3,1,0
4,1.0,0.0,0.0,0.0,0.0,0.0,288.0,233.0,0.0,0.0,...,2.0,-1478.0,2307.0,3785.0,51.0,1.0,0.0,0,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,2.0,4421.0,9650.0,5229.0,45.0,0.0,0.0,0,3,0
678,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,0.0,-20.0,21.0,41.0,45.0,0.0,0.0,3,3,0
679,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,0.0,-2.0,255.0,445.0,47.0,0.0,0.0,2,3,0
680,0.5,0.0,0.0,0.0,0.0,0.0,357.5,236.0,0.0,0.0,...,0.0,-3728.0,158.0,3886.0,86.0,1.0,0.0,1,5,2


In [42]:
columns = df1.columns
remove = ('pos','teams','id','total_points')
a = set(columns)
b = a.difference(remove)
columns = list(b)

In [43]:
df1.to_csv('aavash.csv')

In [44]:
df1.loc[:,columns] = StandardScaler().fit_transform(df1.loc[:, columns])


In [45]:
df1

,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,element_0,fixture_0,goals_conceded_0,goals_scored_0,...,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4,pos,teams,total_points
0,2.846618,-0.169008,4.527569,3.034911,3.954004,2.338271,0.281407,-0.337308,-0.484822,-0.16126,...,-0.400587,-1.144913,-0.233508,1.726346,0.282392,-0.699345,-0.144769,0,21,2
1,0.113593,-0.169008,-0.190234,0.066847,-0.252908,-0.288930,1.159391,-1.094032,-0.484822,-0.16126,...,3.114861,-0.104206,-0.250281,-0.311929,-0.077088,-0.699345,-0.144769,3,4,2
2,4.160572,-0.169008,4.527569,3.153633,3.954004,-0.267913,1.073734,0.041054,-0.484822,-0.16126,...,-0.400587,-0.104167,-0.241784,-0.280663,-0.256828,-0.699345,-0.144769,2,16,3
3,-0.569663,-0.169008,-0.190234,-0.526766,-0.252908,-0.373001,0.736460,-1.850757,-0.484822,-0.16126,...,-0.400587,-0.104167,-0.241784,-0.280663,-0.256828,-0.699345,-0.144769,3,1,0
4,-0.044081,-0.169008,-0.190234,-0.526766,-0.252908,-0.373001,-0.377081,-1.094032,-0.484822,-0.16126,...,0.603827,-0.161632,-0.165971,-0.030237,0.102652,1.429910,-0.144769,0,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,-0.306872,-0.169008,-0.190234,-0.526766,-0.252908,-0.373001,-0.005008,0.041054,-0.484822,-0.16126,...,0.603827,0.068032,0.105322,0.078031,-0.436568,-0.699345,-0.144769,0,3,0
678,-0.306872,-0.169008,-0.190234,-0.526766,-0.252908,-0.373001,-0.005008,0.041054,-0.484822,-0.16126,...,-0.400587,-0.104868,-0.250429,-0.310954,-0.436568,-0.699345,-0.144769,3,3,0
679,-0.306872,-0.169008,-0.190234,-0.526766,-0.252908,-0.373001,-0.005008,0.041054,-0.484822,-0.16126,...,-0.400587,-0.104167,-0.241784,-0.280663,-0.256828,-0.699345,-0.144769,2,3,0
680,-0.306872,-0.169008,-0.190234,-0.526766,-0.252908,-0.373001,-0.005008,0.041054,-0.484822,-0.16126,...,-0.400587,-0.249230,-0.245368,-0.022664,3.248104,1.429910,-0.144769,1,5,2


In [46]:
X=df1.drop(['id' ,'total_points' ,'pos','teams'],axis ='columns') 
Y= df1.loc[:, {'total_points'}]


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2 )




In [47]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=20, random_state=0)

model.fit(X_train ,Y_train)

<ipython-input-47-508aef32fc27>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train ,Y_train)


RandomForestRegressor(max_depth=20, random_state=0)

In [48]:
prediction = model.predict(X_test)

In [49]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test,prediction)

5.771167527296857

In [50]:
pred_training = model.predict(X_train)
mean_squared_error(Y_train,pred_training)

0.48064275096671466

In [51]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test,prediction)


1.2992700729927007

In [52]:
mean_squared_error(Y_test,prediction)

5.771167527296857

In [53]:
with open('model_pickle','wb') as file:
    pickle.dump(model,file)

